# Generate 3D Hand Joints from 2D Keypoints by RANSAC and Triangulation

## What is RANSAC?

RANSAC (RANdom SAmple Consensus) is an iterative method to estimate parameters of a mathematical model from a set of observed data that contains outliers. It is a non-deterministic algorithm in the sense that it produces a reasonable result only with a certain probability, with this probability increasing as more iterations are allowed.


## What is Triangulation?

Triangulation is the process of determining the location of a point by measuring angles to it from known points at either end of a fixed baseline, rather than measuring distances to the point directly. The point can then be fixed as the third point of a triangle with one known side and two known angles.

## How to use RANSAC and Triangulation to estimate 3D Hand Joints?

1. Create the list of candidate 3D hand joints.
   - For hand joint $J_i$, we have $N$ 2D keypoints from $N$ camera views.
     - For each pair of keypoints $(k_{i,1}, k_{i,2})$ from different camera views (camera1, camera2), we can triangulate the 3D point $P_i$.
   - We can get $N(N-1)/2$ 3D points for each joint $J_i$.
   - Repeat this process for all joints to get a list of candidate 3D hand joints.
   - Note that some of the 3D points may be outliers or incorrect due to noise or occlusions.
   - We can use RANSAC to find the best estimate of the 3D point for each joint.
   
   In our setting, we have 8 camera views, so we can get 28 candidate 3D points for each joint. And there are 21 joints in total, so the merged array of candidate 3D points has the shape of (21, 28, 3).
2. Apply RANSAC for Each Joint:
   - We will use the 2D projection error as the distance metric for RANSAC.
   - For each joint $J_i$:
     - Initialize RANSAC parameters (number of iterations, threshold, minimum inliers).
     - Iterate RANSAC:
       - Randomly select a subset of 3D points.
       - Fit a model (in this case, the mean 3D point) to the subset.
       - Project the 3D model point back to 2D for each camera.
       - Calculate the 2D projection error for all points.
       - Identify inliers (points with projection error within the threshold).
       - If the number of inliers is greater than the current best model, update the best model.
     - Output the best estimate for joint $J_i$.
   - Here, the best estimate is the 3D point that has the most inliers in the RANSAC process.
   - We can repeat this process for all joints to get the final 3D hand joints $J_{3d}$, which is a (21, 3) array.
3. Fitting the MANO Model to RANSAC Estimated 3D Hand Joints:
   
   Although the RANSAC estimated 3D hand joints are more robust to outliers, they may not be the reasonable 3D hand joints for the hand pose. We can use the MANO model to fit the RANSAC estimated 3D hand joints to get the final MANO hand pose.

   - Initialize the MANO pose by copying the global translation from the RANSAC estimated 3D hand joints and setting the rest of the parameters to zero.
   - We use below three loss functions to fit the MANO model to the RANSAC estimated 3D hand joints:
     - **3D Joint Loss**: The L2 loss between the MANO model joints and the RANSAC estimated 3D hand joints.
     - **2D Projection Loss**: The L2 loss between the 2D projection of the MANO model joints and the 2D keypoints.
     - **Regularization Loss**: The L2 loss between the MANO pose parameters and the initial pose parameters.
   - We can use an optimization algorithm (e.g., Adam optimizer) to minimize the sum of the three loss functions.
   - The final MANO pose parameters are the estimated hand pose.
   - We can use the MANO model to generate the final 3D hand joints, 2D keypoints, and the hand mesh. And we can visualize the hand pose in both 3D and 2D.

## Practice:

We have extracted the synchronized frames from 8 camera views. We can use the mediapipe to get the 2D keypoints from the frames. Then we can use the above method to estimate the 3D hand. Follow the below steps to practice the method:

1. Picke one frame from the recording sequence under your name.
2. Run mediapipe to get 2D keypoints from 8 camera views.
3. Triangulate the 3D points from the 2D keypoints.
4. Apply RANSAC to estimate the 3D hand joints.
5. Fit the MANO model to the RANSAC estimated 3D hand joints for the final hand pose.
6. Visualize the hand pose in 3D and 2D.